In [2]:
import torch
import numpy as np
import pandas as pd
import copy
import torch.nn.functional as F
import scipy.sparse as sp
from torch_geometric.nn import GCNConv,GATConv,SAGEConv
from torch_geometric.datasets import Planetoid
from torch.nn import Linear
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [ ]:
# 对样本进行归一化，而不是特征
def Scale(X):
    raw = X
    scaler = StandardScaler()
    scaler.fit(X.T)
    X = pd.DataFrame(scaler.transform(X.T)).T
    X.columns = raw.columns
    X.index = raw.index
    return X

In [3]:
def ProssGSEA(csv):
    dataframe = pd.read_csv(csv,sep=",")
    dataframe = dataframe.pivot(index = "Name",columns=["Term"],values=["NES"])
    dataframe.columns = dataframe.columns.droplevel(0)
    dataframe.index.rename(None, inplace=True)
    dataframe = dataframe.dropna(axis=1, how='all') ## 去除所有都是NaN的列
    return dataframe

In [5]:
ReactomePathways_file = "/public/ref/Msigdb/Reactome/useInfo/Human.ReactomePathways.list"
ReactomePathways_label = pd.read_csv(ReactomePathways_file,header=None,sep='\t')
#print (ReactomePathways_label.iloc[0:1,0:1])
ReactomePathways_label.columns = ["Name","ID"]
ReactomePathways_label_dict = dict(zip(ReactomePathways_label["Name"],ReactomePathways_label["ID"])) ## Name : R-HSA*
ReactomePathways_label_rev_dict = dict(zip(ReactomePathways_label["ID"],ReactomePathways_label["Name"])) # R-HSA*: Name
print (ReactomePathways_label_rev_dict["R-HSA-9694493"])
print (ReactomePathways_label_dict["Maturation of protein E_9694493"])
print (ReactomePathways_label_dict['Hereditary fructose intolerance'])


Maturation of protein E_9694493
R-HSA-9694493
R-HSA-5657560


In [6]:
#GSEA_file = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GSEA/GSEA_5tissue50.csv"
GSEA_file = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GSEA/GSEA_all.csv"
#GSEA_score = pd.read_csv(GSEA_file,header=0,sep=',')
#print (GSEA_score.iloc[0:3,0:3])
#GSEA_score["Term"] = GSEA_score["Term"].map(ReactomePathways_label_dict)  ## 某一列替换为对应字典值
#GSEA_score = GSEA_score.pivot(index="Name",columns="Term",values="NES")
#GSEA_score.index.rename(None, inplace=True)
#GSEA_score = GSEA_score.dropna(axis=1, how='all')
GSEA_score =ProssGSEA(GSEA_file)
#GSEA_score = Scale(GSEA_score)
print (GSEA_score.shape)
print (GSEA_score.iloc[0:3,0:5])
#GSEA_score_ndarry =  GSEA_score.values
GSEA_score_ndarry = preprocessing.scale(GSEA_score.values,axis=1)
print (GSEA_score.values[[2],[3]])


(17382, 2582)
Term                      R-HSA-1059683  R-HSA-109581  R-HSA-109582  \
GTEX-1117F-0226-SM-5GZZ7       0.448864      0.405386      0.323341   
GTEX-1117F-0426-SM-5EGHI       0.433999      0.412742      0.294845   
GTEX-1117F-0526-SM-5EGHJ       0.455226      0.404896      0.320925   

Term                      R-HSA-109606  R-HSA-109703  
GTEX-1117F-0226-SM-5GZZ7      0.396594      0.360710  
GTEX-1117F-0426-SM-5EGHI      0.408149      0.344490  
GTEX-1117F-0526-SM-5EGHJ      0.397948      0.332731  
[0.39794807]


In [ ]:
print (GSEA_score.loc["GTEX-1117F-0426-SM-5EGHI"])

In [7]:
edges_file = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/Human.pathway.Hierarchy.txt"
edges_pd = pd.read_csv(edges_file,header=None,sep='\t')
#edges_pd.columns = ["Edge1","Edge2","Type","TopLevel"]
edges_pd.columns = ["Edge1","Edge2"]
#print (edges_pd["Edge1"])
#print (type(list(edges_pd['Edge1'])))
print (edges_pd.shape)
edges_pd = edges_pd.loc[(edges_pd["Edge1"].isin(GSEA_score.columns)) &  edges_pd["Edge2"].isin(GSEA_score.columns)]
edges_pd.reset_index()
print (edges_pd.shape)
edges_list = sorted(pd.concat([edges_pd['Edge1'],edges_pd['Edge2']]).unique().tolist())
edges_dict = dict(zip(edges_list,range(len(edges_list))))
edges_dict_rev = dict(zip(range(len(edges_list)),edges_list))
print ("R-HSA-109582" in edges_list)
print (edges_pd.at[10,"Edge1"])
print (edges_dict[edges_pd.at[10,"Edge1"]])
#print (edges)
edges = list()
print (edges_pd.at[97,"Edge1"])
for i in range(edges_pd.shape[0]):
    #print (edges_pd.at[i,"Edge1"])
    #print (i)
    edges.append([edges_dict[edges_pd.iat[i,0]],edges_dict[edges_pd.iat[i,1]]])
    #print (edges)
edges = torch.tensor(edges,dtype = torch.int64).T
print (edges)
print (edges.shape)

(2647, 2)
(2600, 2)
True
R-HSA-109582
2
R-HSA-1187000
tensor([[   1,    1,    1,  ..., 2579, 2580, 2580],
        [   3,  359, 1240,  ..., 2177,  362, 2581]])
torch.Size([2, 2600])


In [ ]:
print (ReactomePathways_label_rev_dict[edges_dict_rev[2]])
torch.save(edges,"/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/pathway_edges.pt")

In [ ]:
print (GSEA_score_ndarry[[2],[5]])

In [4]:
# 准备sample对应的tissue和对应的label 标签 ()
#label_file = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GSEA/GSEA_5tissue50.label.txt"
label_file = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GSEA/GTEx.sample_label.txt"
sample_labels = pd.read_csv(label_file,header=0,sep="\t")
sample_full_labels = sorted(list(set(sample_labels["SMTS"].tolist()))) ## 唯一值
sample_full_labels_dict = dict(zip(sample_full_labels,range(len(sample_full_labels)))) ## label对应的index
list_labels = []
print (sample_full_labels_dict)
sample_dict = dict(zip(sample_labels["SAMPID"],sample_labels["SMTS"]))
print (sample_full_labels_dict[sample_dict["GTEX-1GF9W-1326-SM-7P8PX"]])
for x in range(len(sample_full_labels)):
    list_labels.append([x])
list_labels_tensor = torch.tensor(list_labels,dtype=torch.int64)
print (list_labels_tensor[sample_full_labels_dict[sample_dict["GTEX-1GF9W-1326-SM-7P8PX"]]])

{'Adipose Tissue': 0, 'Adrenal Gland': 1, 'Bladder': 2, 'Blood': 3, 'Blood Vessel': 4, 'Bone Marrow': 5, 'Brain': 6, 'Breast': 7, 'Cervix Uteri': 8, 'Colon': 9, 'Esophagus': 10, 'Fallopian Tube': 11, 'Heart': 12, 'Kidney': 13, 'Liver': 14, 'Lung': 15, 'Muscle': 16, 'Nerve': 17, 'Ovary': 18, 'Pancreas': 19, 'Pituitary': 20, 'Prostate': 21, 'Salivary Gland': 22, 'Skin': 23, 'Small Intestine': 24, 'Spleen': 25, 'Stomach': 26, 'Testis': 27, 'Thyroid': 28, 'Uterus': 29, 'Vagina': 30}
13
tensor([13])


In [ ]:
GSEA_score_ndarry[[2]]

In [ ]:
#print (GSEA_score.loc["GTEX-117YW-2226-SM-5N9DB"])
print (GSEA_score.shape)


In [8]:
import datetime
from torch_geometric.data import Data
uniq_sample = sorted(list(set(GSEA_score.index)))
train_dataset = list()
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
for sampleIndex,sampleName in enumerate(uniq_sample):
    #print (sampleName)
    sampleValue = GSEA_score_ndarry[sampleIndex]
    sampleValue = sampleValue.reshape(sampleValue.shape[0],1)
    #print (list_labels_tensor[sample_full_labels_dict[sample_dict[sampleName]]])
    sample_node_feature = Data(x=torch.tensor(sampleValue,dtype = torch.float32),y=list_labels_tensor[sample_full_labels_dict[sample_dict[sampleName]]],edge_index = edges)
    train_dataset.append(sample_node_feature)
    #break
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
print (len(train_dataset))

2023-09-24  10:39:28
2023-09-24  10:39:29
17382


In [14]:
print (train_dataset[0])

Data(x=[2582, 1], edge_index=[2, 2600], y=[1])


In [ ]:
#a = GSEA_score.loc["GTEX-1GF9W-1326-SM-7P8PX"]
#a = torch.tensor(a,dtype = torch.float32)
print (train_dataset[0])
print (train_dataset[0].x)
print (train_dataset[0].y)
#print (type(a))

In [9]:
from torch.nn import Linear
from torch_geometric.nn import GraphConv

class GNN(torch.nn.Module):
    def __init__(self, num_node_features,hidden_channels,num_classes):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels,num_classes)
        
    def forward(self, x, edge_index, batch):
        # 1. 获得节点嵌入
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        
        # 2. Readout layer
        x = global_mean_pool(x, batch)   # [batch_size, hidden_channels]
        
        # 3. 分类器
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x

In [10]:
#model = GNN(num_node_features=2582,hidden_channels=64,num_classes=len(sample_full_labels_dict.keys()))
model = GNN(num_node_features=1,hidden_channels=64,num_classes=31)
print (model)

GNN(
  (conv1): GraphConv(1, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=31, bias=True)
)


In [11]:
from torch_geometric.nn import global_mean_pool
from torch_geometric.loader import DataLoader
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
torch.set_num_threads(48)
print (train_dataset[0])
train_dataset1 = train_dataset[0:17000]
train_loader = DataLoader(train_dataset1, batch_size=30,shuffle=True)
def train():
    model.train()
    for data in train_loader:
        #print (data.y)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index,data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
def test(loader):
    model.eval()
    correct = 0
    for data in loader:                            # 批遍历测试集数据集。
        out = model(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)   # 使用概率最高的类别
        #print (pred,data.y)
        correct += int((pred == data.y).sum())           # 检查真实标签
    return correct / len(loader.dataset)

model.train()
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))

for epoch in range(1, 301):
    train()
    print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
    train_acc = test(train_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')

Data(x=[2582, 1], edge_index=[2, 2600], y=[1])
2023-09-24  10:39:38
2023-09-24  10:40:13
Epoch: 001, Train Acc: 0.1536
2023-09-24  10:40:56
Epoch: 002, Train Acc: 0.3124
2023-09-24  10:41:40
Epoch: 003, Train Acc: 0.4195
2023-09-24  10:42:17
Epoch: 004, Train Acc: 0.4651
2023-09-24  10:43:01
Epoch: 005, Train Acc: 0.4665
2023-09-24  10:43:45
Epoch: 006, Train Acc: 0.5003
2023-09-24  10:44:27
Epoch: 007, Train Acc: 0.5021
2023-09-24  10:45:10
Epoch: 008, Train Acc: 0.5197
2023-09-24  10:45:54
Epoch: 009, Train Acc: 0.5412
2023-09-24  10:46:37
Epoch: 010, Train Acc: 0.5584
2023-09-24  10:47:20
Epoch: 011, Train Acc: 0.5542
2023-09-24  10:48:02
Epoch: 012, Train Acc: 0.5913
2023-09-24  10:48:45
Epoch: 013, Train Acc: 0.5724
2023-09-24  10:49:27
Epoch: 014, Train Acc: 0.5487
2023-09-24  10:50:09
Epoch: 015, Train Acc: 0.6147
2023-09-24  10:50:53
Epoch: 016, Train Acc: 0.6081
2023-09-24  10:51:39
Epoch: 017, Train Acc: 0.6438
2023-09-24  10:52:24
Epoch: 018, Train Acc: 0.6665
2023-09-24  10

Epoch: 160, Train Acc: 0.9368
2023-09-24  12:35:47
Epoch: 161, Train Acc: 0.9339
2023-09-24  12:36:31
Epoch: 162, Train Acc: 0.9362
2023-09-24  12:37:15
Epoch: 163, Train Acc: 0.9312
2023-09-24  12:37:58
Epoch: 164, Train Acc: 0.9256
2023-09-24  12:38:42
Epoch: 165, Train Acc: 0.9226
2023-09-24  12:39:25
Epoch: 166, Train Acc: 0.9398
2023-09-24  12:40:09
Epoch: 167, Train Acc: 0.9368
2023-09-24  12:40:54
Epoch: 168, Train Acc: 0.9229
2023-09-24  12:41:37
Epoch: 169, Train Acc: 0.9277
2023-09-24  12:42:21
Epoch: 170, Train Acc: 0.9356
2023-09-24  12:43:04
Epoch: 171, Train Acc: 0.9208
2023-09-24  12:43:48
Epoch: 172, Train Acc: 0.9340
2023-09-24  12:44:32
Epoch: 173, Train Acc: 0.9377
2023-09-24  12:45:15
Epoch: 174, Train Acc: 0.9361
2023-09-24  12:45:59
Epoch: 175, Train Acc: 0.9396
2023-09-24  12:46:43
Epoch: 176, Train Acc: 0.9392
2023-09-24  12:47:26
Epoch: 177, Train Acc: 0.9330
2023-09-24  12:48:10
Epoch: 178, Train Acc: 0.9203
2023-09-24  12:48:55
Epoch: 179, Train Acc: 0.9388
2

In [1]:
print (train_loader[0].batch)

NameError: name 'train_loader' is not defined

In [ ]:
def test(loader):
    model.eval()
    correct = 0
    for data in loader:                            # 批遍历测试集数据集。
        out = model(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)   # 使用概率最高的类别
        print (pred,data.y)
        correct += int((pred == data.y).sum())           # 检查真实标签
    return correct / len(loader.dataset)

In [13]:
# 测试集
model.eval()
test_data = train_dataset[16000:]
print (len(test_data))
test_loader = DataLoader(test_data, batch_size=10,shuffle=True)
test_acc = test(test_loader)
#print (test_acc.)
print(f'Test Acc: {test_acc:.4f}')

1382
Test Acc: 0.9255


In [ ]:
model.eval()
#print (test_loader)
data = test_data[1]
#print (data)
out = model(data.x, data.edge_index,data.batch) # 一次前向传播
print (out)
pred = out.argmax(dim=1)
print (pred)

In [12]:
# 保存模型
saved_dict = {
 'model': model.state_dict(),
 'opt': optimizer.state_dict()
}
torch.save(saved_dict, '/public/home/liujunwu/workdir/scripts/GNN_Reactome/GTEx_exp/GTEx.nomalizeSample.axis1.pth.tar')

In [ ]:
#TCGA_GSEA_score = TotalSample_GSEA.csv 
TCGA_GSEA_score = pd.read_csv("/public/home/liujunwu/workdir/scripts/GNN_Reactome/TCGA_exp/GSEA/TotalSample_GSEA.csv",header=0,sep=',')
TCGA_GSEA_score = TCGA_GSEA_score.pivot(index="Name",columns="Term",values="NES")
TCGA_GSEA_score.index.rename(None, inplace=True)
TCGA_GSEA_score = TCGA_GSEA_score.dropna(axis=1, how='all')
print (TCGA_GSEA_score.shape)
print (TCGA_GSEA_score.iloc[0:3,0:5])
TCGA_GSEA_score_ndarry = preprocessing.scale(TCGA_GSEA_score.values,axis=1)
print (TCGA_GSEA_score_ndarry[[2],[3]])

In [ ]:
## 随机切割 数据
from torch.utils.data import random_split
TCGA_train_dataset, TCGA_test_dataset = random_split(
    dataset=tuning_dataset,
    lengths=[365, 365],
    generator=torch.Generator().manual_seed(0)
)
print (len(TCGA_train_dataset))
print (len(TCGA_test_dataset))